In [4]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pydicom
import imageio
import os
from glob import glob
import nibabel as nib
from tqdm import tqdm
import json
import nibabel as nib
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy.ndimage import zoom
import nibabel as nib
from glob import glob
from tqdm import tqdm
import pandas as pd
import scipy.ndimage
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [2]:

data_dir='../../data/250513 C MR/'
save_dir='../../data/mri_nii/'
df=pd.read_csv('../../data/filter_mri_df.csv')


In [5]:
folder_list = [os.path.join(data_dir, pid) for pid in df['ID']]

for folder in tqdm(folder_list):
    dicom_paths = sorted(glob(folder + '/*.dcm'))
    slices = []

    for dicom_file in dicom_paths:
        dcm = pydicom.dcmread(dicom_file)
        img = dcm.pixel_array.astype(np.float32)
        slope = float(getattr(dcm, 'RescaleSlope', 1))
        intercept = float(getattr(dcm, 'RescaleIntercept', 0))
        img = img * slope + intercept
        slices.append(img)

    volume = np.stack(slices, axis=0)  # (Z, H, W)

    # ✅ 정규화
    vmin = np.percentile(volume, 5)
    vmax = np.percentile(volume, 100)
    volume = np.clip(volume, vmin, vmax)
    volume = (volume - vmin) / (vmax - vmin)
    volume = volume * 2 - 1

    # ✅ spacing 추출
    z_spacing = float(getattr(dcm, 'SliceThickness', 1))  # Z 방향은 유지
    y_spacing, x_spacing = map(float, getattr(dcm, 'PixelSpacing', [1, 1]))

    # ✅ 리샘플링 비율 계산 (Z는 그대로 → 1.0)
    target_yx_spacing = (0.5, 0.5)
    zoom_factors = [1.0, y_spacing / target_yx_spacing[0], x_spacing / target_yx_spacing[1]]

    # ✅ (Z, H, W) → (Z, new_H, new_W)
    volume = scipy.ndimage.zoom(volume, zoom=zoom_factors, order=1)
    volume = volume.astype(np.float32)

    # 저장
    patient_id = os.path.basename(folder).replace('.dcm', '')
    save_path = os.path.join(save_dir, f"{patient_id}.nii.gz")
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    nifti_img = nib.Nifti1Image(volume, affine=np.eye(4))
    nib.save(nifti_img, save_path)

100%|██████████| 306/306 [04:17<00:00,  1.19it/s]
